## 一、概览

数据来源：广州链家网二手房交易记录（2012年5月3日至2018年6月12日）；

数据获取方式：基于R语言rvest包的网络爬虫；

数据存储方式：csv文件；

数据量：二手房交易记录23018条 ；

分析、处理工具：R语言；

处理、清洗方法：格式转换、异常值剔除等；

绘图工具：R语言。

## 二、数据获取

### （一）获取二级区域访问地址列表

链家广州二手房交易记录的访问地址为:[https://gz.lianjia.com/chengjiao/](https://gz.lianjia.com/chengjiao/)。可以看到，有23018套房屋交易记录（截止到**2018年6月12日**）。![chengjiao](mywork/images/chengjiao.png)

每一页显示30条交易记录，访问的链接地址直接在访问地址后面添加后缀/pg + （页码）：![url_chengjiao](mywork/images/url_chengjiao.png)

但是，链家仅提供前100页可访问记录，每页显示30条记录。因此，无法获取全部交易记录。![pages_chengjiao.png](mywork/images/pages_chengjiao.png)。

我们发现，页面提供按一级区域（如天河、越秀、荔湾等）和二级区域（如白云山风景区、岑村、车陂长兴、东圃等）分别进行查询，且二级区域一般交易记录不超过100页，因此考虑使用二级区域分别查询汇总，来获取全部交易数据。![region.png](mywork/images/region.png)

接下来，可以结合rvest包和CSS选择器来进行二级区域链接获取。R代码如下：


In [1]:
# 导入相关爬虫包
library(rvest)
library(curl)

urlbase <- 'https://gz.lianjia.com/'
url0 <- paste(urlbase,'/chengjiao/',sep="")
web <- read_html(url0)

#一级区域
html1 <- web %>% html_nodes(".m-filter .position dl:last-of-type [data-role='ershoufang'] div:first-of-type a")
list1 <- html1 %>% html_attr("href")
district1 <- html1 %>% html_text()

url1 <- paste(urlbase, list1, sep="")
url2 <- NULL
district2 <- NULL

#二级区域
for(i in 1:length(url1)) {
    web <- read_html(url1[i])
    htmltemp <- web %>% html_nodes(".m-filter .position dl:last-of-type [data-role='ershoufang'] div:last-of-type a")
    listtemp <- htmltemp %>% html_attr("href")
    districttemp <- htmltemp %>% html_text()
    districttemp <- paste(district1[i], districttemp, sep="/")
    district2 <- c(district2, districttemp)
    urltemp <- paste(urlbase, listtemp, sep="")
    url2 <- c(url2, urltemp)
}

Loading required package: xml2


In [2]:
url2[1:5]
district2[1:5]

[1] "https://gz.lianjia.com//chengjiao/baiyunshanfengjingqu/"
[2] "https://gz.lianjia.com//chengjiao/cencun/"              
[3] "https://gz.lianjia.com//chengjiao/chebei/"              
[4] "https://gz.lianjia.com//chengjiao/changxing1/"          
[5] "https://gz.lianjia.com//chengjiao/dongpu/"

[1] "天河/白云山风景区" "天河/岑村"         "天河/车陂"        
[4] "天河/长兴"         "天河/东圃"

### （二） 获取交易记录

首先，需要获取该二级区域交易总数，然后记录总的页数，拼接访问地址。比如，天河/天河公园总共有319条交易记录，则 $\lceil 319/30 \rceil = 11$（向上取整），因此需要访问$pg1 \sim pg11$。

接下来，需要分析每页的交易记录，可以爬取的内容，如下：

- 区域（district）：天河/天河公园

- 标题（title）：佳福阁 2室2厅 89平米

- 交易时间（deal_date）：2018.04.17

- 房屋信息（house_Info）：南 | 其他 | 无电梯

- 房屋成交价格（total_price）：306万

- 房屋楼层信息（position_Info）：低楼层(共9层) 1998年建塔楼

- 房屋单价（unit_price）：34383元/平

- 房屋位置信息（deal_HouseInfo）：房屋满五年 距5号线科韵路638米

- 挂牌价（quoted_price）：挂牌320万

- 交易周期（deal_cycle）：成交周期234天

- 页面地址（href）：https://gz.lianjia.com/chengjiao/GZ0003010863.html

R代码如下：

```R
#开始逐个二级区域检索
total <- NULL
lianjia <- NULL
print(length(url2))
for(i in 1:length(url2)) {
    print(i)
    web <- read_html(url2[i])
    totaltemp <- web %>% html_nodes(".leftContent .total span") %>% html_text()
    totaltemp <- as.numeric(totaltemp)
    #各二级区域总数
    total <- c(total, totaltemp)
    #该二级区域页码
    num <- ceiling(totaltemp/30)
    j <- 0
    try <- 0
    while(j < num) {
        print(j)
        j = j + 1
        Sys.sleep(5)
        url <- paste(url2[i], "pg", j, sep="")
        #web <- read_html(url)
        web <- read_html(curl(url, handle = curl::new_handle("useragent" = "Mozilla/5.0")))
        list <- web %>% html_nodes('.content .leftContent .listContent li')
        href <- list %>% html_nodes(".img") %>% html_attr("href")
        info <- list %>% html_nodes(".info")
        title <- info %>% html_nodes(".title") %>% html_text()
        house_Info <- info %>% html_nodes(".address .houseInfo") %>% html_text()
        deal_date <- info %>% html_nodes(".address .dealDate") %>% html_text()
        total_price <- info %>% html_nodes(".address .totalPrice") %>% html_text()
        position_Info <- info %>% html_nodes(".flood .positionInfo") %>% html_text()
        unit_price <- info %>% html_nodes(".flood .unitPrice") %>% html_text()
        deal_HouseInfo <- info %>% html_nodes(".dealHouseInfo .dealHouseTxt") %>% html_text()
        deal_HouseInfoPro <- NULL
        
        l = 0
        for(k in 1:length(list)) {
            if(length(info[k] %>% html_nodes(".dealHouseInfo")) == 0){
                deal_HouseInfoPro <- c(deal_HouseInfoPro , "")
            } else {
                l = l + 1
                deal_HouseInfoPro <- c(deal_HouseInfoPro , deal_HouseInfo[k])
            }
        }
        quoted_price <- info %>% html_nodes(".dealCycleeInfo .dealCycleTxt span:first-of-type") %>% html_text()
        deal_cycle<- info %>% html_nodes(".dealCycleeInfo .dealCycleTxt span:last-of-type") %>% html_text()
        if(length(title) != 0){
            try <- 0
            res <- data.frame(district2[i], title, deal_date, house_Info, total_price, 
            position_Info, unit_price, deal_HouseInfoPro , quoted_price, deal_cycle, href, totaltemp)
            rbind(lianjia, res)
            if(i==1 && j == 1){
                write.table(res,file="gz_esf_deal.csv",sep=",",quote=F,row.names=F,col.names=T)
            } else {
                write.table(res,file="gz_esf_deal.csv",sep=",",append=T,quote=F,row.names=F,col.names=F)
            }
        } else {
            if(try < 5){
                # 获取数据失败，重试一次
                j <- j - 1
                try <- try + 1
                cat("等待重连，第",try,"次... ...\n")
                Sys.sleep(20)
            } else {
                cat("已经重连",try,"次，退出！\n")
                break
            }
        }
    }
    if(try >= 5){
        break
    }
}
```

In [3]:
head(data[, 1:12])

ERROR: Error in `[.data.frame`(data, , 1:12): undefined columns selected


### （三）交易数据清洗和数据切分

这块的工作主要是将不符合要求的记录数据补齐或是剔除，对符合要求的记录进行格式转换、数据切分和重新整合。例如：
- 交易时间字符串转换为时间格式
```R
#时间格式转换
data$deal_date <- as.Date(data[,3], "%Y.%m.%d")
```
- 将标题切分为小区、户型和面积三块
```R
title <- unlist(strsplit(data[,2],split=" "))
```
- 针对房屋信息，有几种格式：
    + 南 北 | 精装<U+00A0>| 无电梯
    + 南 | 有电梯，③南 北 | 其他
   
  我们需要切分，并且根据切分后数目补齐（填入NA）。
 
数据清洗、转换、切分和整合的R代码如下：

In [ ]:
# 载入字符串处理包
library(stringr)

# 读取csv数据
data <- read.table("gz_esf_deal.csv", header = T, colClasses = c("character"), sep=",")

# 数据总量
totalnum <- length(data[, 1])

# 规范命名
names(data)[1] = "district"

# 时间格式转换
data$deal_date <- as.Date(data[, 3], "%Y.%m.%d")

# 标题切分出小区、户型、面积
title <- unlist(strsplit(data[, 2], split = " "))
num <- seq(from = 1, to = totalnum*3, by = 3)
data$block <- title[num]
data$block <- factor(data$block)
data$layout <- title[num + 1]
data$layout <- factor(data$layout)
area <- title[num + 2]
area <- sub(pattern = "平米", replacement = "", area)

# 字符串转换为数值型
data$area <- as.numeric(area)
data$house_Info <- str_trim(sub(pattern = "<U\\+00A0>", replacement = "", data[, 4]))
orientation <- NULL
decoration <- NULL
elevator <- NULL

# houseInfo包含朝向、 装修和电梯中的一种或几种，需逐个判断分解
for(i in 1:totalnum) {
	info <- data$house_Info[i]
	infolist <- str_trim(unlist(strsplit(data$house_Info[i], split="\\|")))
	orientation <- c(orientation, infolist[1])

	if(length(infolist) >= 2) {
		decoration <- c(decoration, infolist[2])
	} else {
		decoration <- c(decoration, NA)
	}

	if(length(infolist) >= 3) {
		elevator <- c(elevator, infolist[3])
	} else {
		elevator <- c(elevator, NA)
	}  
}

# 分解部分添加到数据集并标识为因子
data$orientation <- orientation 
data$orientation <- factor(data$orientation)
data$decoration <- decoration
data$decoration <- factor(data$decoration)
data$elevator <- elevator
data$elevator <- factor(data$elevator)

# positionInfo包含楼层、楼高、年代、楼型中的一种或几种，需逐个判断分解
stair <- NULL
loft <- NULL
years <- NULL
type <- NULL
for(i in 1:totalnum) {
	info <- data$position_Info[i]
	infolist <- str_trim(unlist(strsplit(data$position_Info[i], split=" ")))
	loftlist <- str_trim(unlist(strsplit(infolist[1], split="\\("))) 
	stair <- c(stair, loftlist[1])
	loft <- c(loft, sub(pattern = "共(.*)层\\)", replacement = "\\1", loftlist[2]))

	if(length(infolist) >= 2) {
		if(str_detect(infolist[2], "年建")) {
			typelist <- str_trim(unlist(strsplit(infolist[2], split="年建")))
			years <- c(years, typelist[1])
			
			if(length(typelist) >= 2) {
				type <- c(type, typelist[2])
			} else {
				type <- c(type, NA)
			}
		} else {
			years <- c(years, NA)
			type <- c(type, infolist[2])
		}
	} else {
		years <- c(years, NA)
		type <- c(type, NA)
	}  
}

# 添加到数据集并标识为因子
data$stair <- stair 
data$stair <- factor(data$stair)
data$loft <- loft
data$loft <- factor(data$loft)
data$years <- years
data$years <- factor(data$years)
data$type <- type
data$type <- factor(data$type)

# dealHouseInfoPro包含交易、地铁中的零种到两种，需逐个判断分解
deal <- NULL
metro <- NULL
data$deal_HouseInfoPro[is.na(data$deal_HouseInfoPro)] <- 0
for(i in 1:totalnum) {
	info <- data$deal_HouseInfoPro[i]

	if(str_detect(info, "房屋满")) {
		deal <- c(deal, sub(pattern = "房屋(满.*年).*", replacement = "\\1", info))
	} else {
		deal <- c(deal, NA)
	}

	if(str_detect(info, "距")){
		metro <- c(metro, sub(pattern = ".*距(.*)", replacement = "\\1", info))
	} else {
		metro <- c(metro, NA)
	}	
}

# 添加到数据集并标识为因子
data$deal <- deal 
data$deal <- factor(data$deal)
data$metro <- metro
data$metro <- factor(data$metro)

# 成交总价转为数值型，便于后续计算
total_price <- sub(pattern = "万", replacement = "", data[, 5])
data$total_price <- as.numeric(total_price)
unit_price <- sub(pattern = "元/平", replacement = "", data[, 7])
data$unit_price <- as.numeric(unit_price)
quoted_price <- sub(pattern = "挂牌(.*)万", replacement = "\\1", data[, 9])
data$quoted_price <- as.numeric(quoted_price)
deal_cycle <- sub(pattern = "成交周期(.*)天", replacement = "\\1", data[, 10])
data$deal_cycle <- as.numeric(deal_cycle)
save(data, file='gz_esf_deal_cleaned.Rdata')

#清洗和切分完成的数据保存
write.table(data, file = "gz_esf_deal_cleaned.csv", sep = ",", quote = F, row.names = F, col.names = T)

In [ ]:
head(data[, 13:24])

In [ ]:
str(data)

### （四）数据清理结果

数据总和：25562条


## 三、数据分析

In [ ]:
max(data$deal_date)

In [ ]:
load('gz_esf_deal_cleaned.Rdata') 

totalnum <- length(data[, 1])
#1、链家每日销售量
day_sale_amount <- with(data, table(deal_date))
barplot(day_sale_amount, xlab="交易时间", ylab="交易量(套)", main="链家广州交易量统计(2012.5.3-2017.7.6)")

In [ ]:
library(plotrix)

In [ ]:
#2、一级区域成交量排行和销售均价
zone <- unlist(strsplit(data$district, split="\\/"))
num <- seq(from=1, to=totalnum*2, by=2)
data$zone <- zone[num]
data$zone <- factor(data$zone)
zone_sales <- with(data, table(zone))
zone_sales <- sort(zone_sales, decreasing=TRUE)

library(plotrix)

# 计算各区域均价
temp_prices <- aggregate(data$unit_price, by=list(zone=data$zone), mean)
zone_unit_prices <- NULL
zone_unit_prices[t(temp_prices["zone"])] <- t(temp_prices["x"])
zone_unit_prices[2] <- round(zone_unit_prices[2], 0)

twoord.plot(lx=c(1:9), ly=zone_sales, lylim=c(0, max(zone_sales)*1.1), 
rylim=c(0, max(zone_unit_prices)*1.1), main="链家广州各区域交易情况", 
xlab="区域", ylab="成交量(套)", rylab="均价(元)", 
rx=c(1:9), ry=zone_unit_prices[names(zone_sales)], 
type=c("bar", "b"), xlim=c(0, 10), xticklab=names(zone_sales))
text(c(1:9), zone_sales+300, zone_sales)
rypos <- round(zone_unit_prices[names(zone_sales)]*max(zone_sales)/max(zone_unit_prices))
text(c(1:9), rypos+600, round(zone_unit_prices[names(zone_sales)]), col="red")

In [ ]:
 table(deal_date)[2]